In [1]:
import os
import json
import pandas as pd
import numpy as np
from collections import defaultdict

In [5]:
path='d:/project/0/src/python/pydata-book-master/ch02/usagov_bitly_data2012-03-16-1331923249.txt'
records = [json.loads(line) for line in open(path)]
records[0]

{u'a': u'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.78 Safari/535.11',
 u'al': u'en-US,en;q=0.8',
 u'c': u'US',
 u'cy': u'Danvers',
 u'g': u'A6qOVH',
 u'gr': u'MA',
 u'h': u'wfLQtf',
 u'hc': 1331822918,
 u'hh': u'1.usa.gov',
 u'l': u'orofrog',
 u'll': [42.576698, -70.954903],
 u'nk': 1,
 u'r': u'http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf',
 u't': 1331923247,
 u'tz': u'America/New_York',
 u'u': u'http://www.ncbi.nlm.nih.gov/pubmed/22415991'}

In [7]:
time_zones = [rec['tz'] for rec in records if 'tz' in rec]
def get_count2(seq):
    counts = defaultdict(int)
    for x in seq:
        counts[x] += 1
    return counts
counts = get_count2(time_zones)
counts['America/New_York']
def top_counts(count_dict, n=10):
    value_key_pairs = [(count, tz) for tz, count in count_dict.items()]
    value_key_pairs.sort()
    return value_key_pairs[-n:]
top_counts(counts)


[(33, u'America/Sao_Paulo'),
 (35, u'Europe/Madrid'),
 (36, u'Pacific/Honolulu'),
 (37, u'Asia/Tokyo'),
 (74, u'Europe/London'),
 (191, u'America/Denver'),
 (382, u'America/Los_Angeles'),
 (400, u'America/Chicago'),
 (521, u''),
 (1251, u'America/New_York')]

In [11]:
frame = pd.DataFrame(records)
tz_counts = frame['tz'].value_counts()
tz_counts[:10]
clean_tz = frame['tz'].fillna('missing')

clean_tz.ix[3536]
clean_tz[clean_tz=='']='unknown'
tz_counts = clean_tz.value_counts()
tz_counts[:10]
results = pd.Series([x.split()[0] for x in frame.a.dropna()])
results[:5]
results.value_counts()[:5]
cframe = frame[frame.a.notnull()]
opt_sys = np.where(cframe['a'].str.contains('Windows'),'windowns','not windows')
opt_sys
by_tz_os = cframe.groupby(['tz',opt_sys])
agg_counts = by_tz_os.size().unstack().fillna(0)
agg_counts[:10]
indexer = agg_counts.sum(1).argsort()
indexer[:10]
count_sub = agg_counts.take(indexer)[-10:]
normed_subset = count_sub.div(count_sub.sum(1), axis=0)